In [1]:
# Import Libraries
import os.path
import numpy as np
from tensorflow.python.platform import gfile
import pypcd
import os
import subprocess
import pandas as pd
from pandas import DataFrame

In [11]:
# Find number of PointClouds
point_dir = '/home/johan/Documents/Alignment/Points_csv/'
#se crea un vector vacio para guardar los nombres de las imágenes.
file_list = []
#se obtienen todos los nombre de las imagenes en la carpeta train que tengan extensión .csv
file_glob = os.path.join(point_dir, '*.csv')
#se organizan en el vector file_list todos las direcciones de las imágenes 
#encontrados dentro de la carpeta train
file_list.extend(gfile.Glob(file_glob))
file_list = np.sort(file_list)
#kind : {‘quicksort’, ‘mergesort’, ‘C’, ‘stable’}
print(len(file_list))
print(type(file_list[0]),file_list[0:8])

451
<class 'numpy.str_'> ['/home/johan/Documents/Alignment/Points_csv/pointscsv00001.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00002.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00004.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00007.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00010.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00013.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00016.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00025.csv']


In [ ]:
# Alignt PCs
#para cada pointcloud dentro del vector file_list
a = 0 
refetotal = []
for point_dir in file_list:
    if a == 0:
        refe = pd.read_csv(point_dir)
        df = DataFrame(refe, columns= ['x', 'y','z'])
        export_csv = df.to_csv ('/home/johan/Documents/Alignment/Points_csv/refe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
        #print ('a0:\n refe',df.shape)
        #print('ICP finished succesfully')
        #print('------------------------------------------------------')
        
    else:
        pointcloud1_path_refe = "/home/johan/Documents/Alignment/Points_csv/refe.csv"
        refe = pd.read_csv(pointcloud1_path_refe)
        refe = np.array(refe)
        #print ("refe", refe.shape) 
        
        pointcloud2_path_in = point_dir
        icp_path="/home/johan/Libraries/libpointmatcher/build/examples"

        #--------------- correr el ejecutable de c++ ------------------
        #runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path_refe,pointcloud2_path_in,'OutPc'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp.wait()
        #-------------leer los datos de devuelve en el terminal------
        #stdout, stderr = runicp.communicate()
        
        #------------------------------------------------------------
        refe2 = pd.read_csv('/home/johan/repos/GitHub/3D-Reconstruction/refe_OutPc.csv')
        refetotal = DataFrame(refe2, columns= ['x','y','z'])
        refetotal = np.array(refetotal)
        #print('refe_OutPc',refetotal.shape, type(refetotal))
        refetotal = refetotal[1:refetotal.shape[0],:]
        #print('refe_OutPc-WOut[x,y,z]',refetotal.shape)
        refetotal_l = np.vstack((refe,refetotal))
        refetotal = DataFrame(refetotal_l) 
        #print('Acumulado',refetotal.shape)
        export_csv = refetotal.to_csv ('/home/johan/Documents/Alignment/Points_csv/refe.csv', index = None, header=True) 
        #print('ICP finished succesfully')
        #print('------------------------------------------------------') 
    
    a = a +1
    print('a:',a)

a: 1
a: 2
a: 3
a: 4
a: 5
a: 6
a: 7
a: 8
a: 9
a: 10
a: 11


/home/johan/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


a: 12
a: 13
a: 14
a: 15
a: 16
a: 17
a: 18
a: 19
a: 20
a: 21
a: 22
a: 23
a: 24
a: 25
a: 26
a: 27
a: 28
a: 29
a: 30
a: 31
a: 32
a: 33
a: 34
a: 35
a: 36
a: 37
a: 38
a: 39
a: 40
a: 41
a: 42
a: 43
a: 44
a: 45
a: 46
a: 47
a: 48
a: 49
a: 50
a: 51
a: 52
a: 53
a: 54
a: 55
a: 56
a: 57
a: 58
a: 59
a: 60
a: 61
a: 62
a: 63
a: 64
a: 65
a: 66
a: 67
a: 68
a: 69
a: 70
a: 71
a: 72
a: 73
a: 74
a: 75
a: 76
a: 77
a: 78
a: 79
a: 80
a: 81
a: 82
a: 83
a: 84
a: 85
a: 86
a: 87
a: 88
a: 89
a: 90
a: 91
a: 92
a: 93
a: 94
a: 95
a: 96
a: 97
a: 98
a: 99
a: 100
a: 101
a: 102
a: 103
a: 104
a: 105
a: 106
a: 107
a: 108
a: 109
a: 110
a: 111
a: 112
a: 113
a: 114
a: 115
a: 116
a: 117
a: 118
a: 119
a: 120
a: 121
a: 122
a: 123
a: 124
a: 125
a: 126
a: 127
a: 128
a: 129
a: 130
a: 131
a: 132
a: 133
a: 134
a: 135
a: 136
a: 137
a: 138
a: 139
a: 140
a: 141
a: 142
a: 143
a: 144
a: 145
a: 146
a: 147
a: 148
a: 149
a: 150
a: 151
a: 152
a: 153
a: 154
a: 155
a: 156
a: 157
a: 158
a: 159
a: 160
a: 161
a: 162
